In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [4]:
# Define some constants
latitude = [40.79736, 41.78701, 30.1444, 25.7738]
longitude = [-73.97785, -87.77166, -97.66876, -80.1936]
cities = ["ny", "il", "tx", "fl"]
start_date = "2016-01-01"
end_date = "2024-03-20"


In [19]:
def getPrediction(city):
    model = tf.keras.models.load_model('model_'+city+'.keras')
    df = pd.read_pickle("./data_cleaned_"+ city +".pkl")
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df.rename(columns={"day": "day_of_year", "tmax_avg":"tmax", "tmin_avg": "tmin", 'prec_om': "prec", 'humi_vc': "humi"})
    # df.info()
    features = ['day_of_year', 'tmax', 'tmin', 'prec', 'humi']
    df = df[features]
    target = 'tmax'

    # Normalize the features
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df[features])

    time_steps = 60  # Use this many days of data to predict the next day's 'tmax'
    # X, y = create_dataset(df_scaled, df_scaled[:, 1], time_steps)
    # split = int(len(X) * 0.75)  # 70% for training

    # # Split the data
    # X_train, X_test = X[:split], X[split:]
    # y_train, y_test = y[:split], y[split:]

    last_days_data = np.array(df[-(time_steps-1):])

    last_days_scaled = scaler.transform(last_days_data)
    last_days_scaled = np.expand_dims(last_days_scaled, axis=0)
    predicted_tmax_scaled = model.predict(last_days_scaled)
    dummy_array = np.zeros((1, len(features))) 
    dummy_array[:, 1] = predicted_tmax_scaled
    inverse_transformed_array = scaler.inverse_transform(dummy_array)
    predicted_tmax = inverse_transformed_array[:, 1]

    print(f"Predicted 'tmax' for {city} for next day: {predicted_tmax[0]}")

In [ ]:
for city in cities:
  getPrediction(city)